In [ ]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
#library(ggplot2)
library(cowplot)

## Doses administered: Reports from provinces and territories

Information on doses administered by dose number and vaccine product is provided by provinces and territories every 4 weeks.

In the last 4 weeks, the majority of doses administered were dose 4 or more.

Data may not be available for some report weeks because:
- data was not submitted for that week (see data availability)
- data quality assurance resulted in negative values
- Data on doses administered should not be used as a substitute for vaccination coverage estimates.
- National data by dose number is incomplete as:
- data on dose 5+ were not available for Nunavut
- data by dose number were not available for Quebec prior to March 13, 2022, and so appears as "Not reported"
- national data excluded doses administered by federal organizations
- Information is subject to change. Numbers may differ in provincial and territorial reports due to the timing of updates and the suppression of values less than 5 for 7 provinces and territories (see limitations).
- Where possible, the number of doses administered includes all doses administered by a province or territory, regardless of where the person receiving the vaccine lives. However, the number of vaccine doses administered may be overestimated due to - -  provincial and territorial partners including doses administered to their residents by other provinces and territories.
- Use caution when comparing data over time. Reporting delays can result in artificial increases in counts while ongoing data quality assurance activities can result in unexpected decreases.

In [ ]:
data <- read_csv("abc/chr.csv")
data<-data|>
rename(numtotal_dose5_admin= 'numtotal_dose5+_admin')

head(data)

## The original cleaned data

In [ ]:
data_canada<- data|>
filter(prename=="Canada")

data2<-data_canada |>
select(- prename, - prfname, - pruid, - product_name)|>
    select(week_end:numtotal_dose5_admin)
head(data2)

Normilized to days data 

In [ ]:
newdata<- data2|>
    mutate(days=as.numeric(week_end)-18615)|>
    select(-week_end)

In [ ]:
newnewdata <- newdata|>
#mutate(days=days)|>
mutate(total=as.vector(numtotal_totaldoses_admin))|>
mutate(dose1=as.vector(numtotal_dose1_admin))|>
mutate(dose2=as.vector(numtotal_dose2_admin))|>
mutate(dose3=as.vector(numtotal_dose3_admin))|>
mutate(dose4=as.vector(numtotal_dose4_admin))|>
mutate(dose5=as.vector(numtotal_dose5_admin))|>
select(-numtotal_totaldoses_admin,-numtotal_dose5_admin,-numtotal_dose4_admin,-numtotal_dose3_admin,-numtotal_dose2_admin,-numtotal_dose1_admin)
head(newnewdata)

Using `newnewdata` to compelete the analysis 

In [ ]:
na_row_total<- nrow(newnewdata[is.na(newnewdata$total),])
na_row_total
na_row_1<- nrow(newnewdata[is.na(newnewdata$dose1),])
na_row_1
na_row_2<- nrow(newnewdata[is.na(newnewdata$dose2),])
na_row_2
na_row_3<- nrow(newnewdata[is.na(newnewdata$dose3),])
na_row_3
na_row_4<- nrow(newnewdata[is.na(newnewdata$dose4),])
na_row_4
na_row_5<- nrow(newnewdata[is.na(newnewdata$dose5),])
na_row_5

In [ ]:
ggplot(newnewdata, aes(x=days,y=dose3))

sa1<- nls(dose3~c*days^z, data=newnewdata, start=c(c=1750000, z=0.27))
sa1

ggplot(newnewdata, aes(x=days,y=dose3))

#sa2<-lm(log(dose3)~log(days), data=newnewdata)
#sa2

## dose 1

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 7)

x<-newnewdata$days
y<-newnewdata$dose1

bbb <- nls(y ~ height/(1 + exp((xmid - x)/scal)),
                           start = list(xmid = 100, scal = 10, height = 5000000))
#bbb
aaa<- coef(bbb)
earlyx<-c(0:100)
ypred<-aaa[3]/(1 + exp((aaa[1] - earlyx)/aaa[2]))

plot<- plot(ypred~earlyx)+
lines(x,predict(bbb))
plot

## dose 2

In [ ]:
x<-newnewdata$days
y<-newnewdata$dose2

options(repr.plot.width = 12, repr.plot.height = 7)
bbb<-nls(y ~ height/(1 + exp((xmid - x)/scal)),
                           start = list(xmid = 100, scal = 10, height = 5000000))
#bbb
aaa<- coef(bbb)
earlyx<-c(0:350)
ypred<-aaa[3]/(1 + exp((coef(bbb)[1] - earlyx)/coef(bbb)[2]))
#ypred
plot(ypred~earlyx)
lines(x,predict(bbb))
#predict(bbb)

## dose 3

In [ ]:
newnewdata3<-newnewdata|>filter(!is.na(dose3))

In [ ]:
ggplot(newnewdata3, aes(days,dose3))+
geom_point(aes(color=age))

In [ ]:
x<-newnewdata3$days
y<-newnewdata3$dose3

options(repr.plot.width = 12, repr.plot.height = 7)
bbb<-nls(y ~ height/(1 + exp((xmid - x)/scal)),
                           start = list(xmid = 400, scal = 10, height = 5000000))
#bbb
aaa<- coef(bbb)
earlyx<-c(0:600)
ypred<-aaa[3]/(1 + exp((coef(bbb)[1] - earlyx)/coef(bbb)[2]))
#ypred
plot(ypred~earlyx)
lines(x,predict(bbb))

## dose 4

In [ ]:
newnewdata4<-newnewdata|>filter(!is.na(dose4))
newnewdata4

In [ ]:
x<-newnewdata4$days
y<-newnewdata4$dose4 


options(repr.plot.width = 12, repr.plot.height = 7)
bbb<-nls(y ~ height/(1 + exp((xmid - x)/scal)),
                           start = list(xmid = 600, scal = 10, height = 5000000))
#bbb
aaa<- coef(bbb)
earlyx<-c(0:900)
ypred<-aaa[3]/(1 + exp((coef(bbb)[1] - earlyx)/coef(bbb)[2]))
#ypred
plot(ypred~earlyx)
lines(x,predict(bbb))

## dose 5

In [ ]:
newnewdata5<-newnewdata|>filter(!is.na(dose5))
print(newnewdata5)
ggplot(newnewdata5, aes(x=days,y=dose5))+
geom_point(aes(color=age))

In [ ]:
x<-newnewdata5$days
y<-newnewdata5$dose5
plot(y~x)
options(repr.plot.width = 12, repr.plot.height = 7)
bbb<-nls(y ~ height/(1 + exp((xmid - x)/scal)), 
                           start = list(xmid = 640, scal = 10, height = 5000000))
#bbb
aaa<- coef(bbb)
earlyx<-c(0:700)
ypred<-aaa[3]/(1 + exp((coef(bbb)[1] - earlyx)/coef(bbb)[2]))
#ypred
plot(ypred~earlyx)
lines(x,predict(bbb))

In [ ]:
x<-newnewdata$days
y<-newnewdata$numtotal_atleast1dose

options(repr.plot.width = 12, repr.plot.height = 7)
bbb<-nls(y ~ height/(1 + exp((xmid - x)/scal)), 
                           start = list(xmid = 100, scal = 10, height = 5000000))
#bbb
aaa<- coef(bbb)
earlyx<-c(-100:600)
ypred<-aaa[3]/(1 + exp((coef(bbb)[1] - earlyx)/coef(bbb)[2]))
#ypred
plot(ypred~earlyx)
lines(x,predict(bbb))

In [ ]:
based on age 
be careful of tne data 

In [ ]:
bbb <- newnewdata|> nls(numtotal_atleast1dose ~ days/(1 + exp((xmid - days)/scal)), 
                            start = list(xmid = 0, scal = 1, height = 100000))
bbb


In [ ]:
nls(numtotal_atleast1dose ~ days, data=newnewdata)

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 7)
ggplot(newnewdata, aes(x=days,y=numtotal_atleast1dose,color=age))+
geom_point()+
ggtitle("at least 1 dose")
ggplot(newnewdata, aes(x=days,y=dose1,color=age))+
geom_point()+
ggtitle("dose 1")
ggplot(newnewdata, aes(x=days,y=dose2,color=age))+
geom_point()+
ggtitle("dose 2")
ggplot(newnewdata, aes(x=days,y=dose3,color=age))+
geom_point()+
ggtitle("dose 3")
ggplot(newnewdata, aes(x=days,y=dose4,color=age))+
geom_point()+
ggtitle("dose 4")
ggplot(newnewdata, aes(x=days,y=dose5,color=age))+
geom_point()+
ggtitle("dose 5")

#glm
#nls 

In [ ]:
nls(formula, data, start, control, algorithm,
    trace, subset, weights, na.action, model,
    lower, upper, ...)
newnewdata

In [ ]:
#?nls
#pivot longer to make it tibble 
#newnewdata
#?glm

In [ ]:
x<-newnewdata$days
y<-newnewdata$numtotal_atleast1dose

bbb<-nls(y ~ height/(1 + exp((xmid - x)/scal)), 
                           start = list(xmid = 100, scal = 10, height = 5000000))
bbb

In [ ]:
coef(bbb)
earlyx<-c(0:100)
ypred<-coef(bbb)[3]/(1 + exp((coef(bbb)[1] - earlyx)/coef(bbb)[2]))
ypred
plot(ypred~earlyx)
lines(x,predict(bbb))

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10)

firstage3<-newnewdata|>
#    mutate(age=as.factor(age))|>
    filter(age=="0–15")

graph3 <- ggplot(firstage3, aes(x=days, y=numtotal_atleast1dose))+
    geom_point(alpha=0.6)+
geom_line()+
    xlab("Date") +
    ylab("Number of New Vaccinations") +
    theme(text = element_text(size = 12))        

graph3

In [ ]:
firstage4<-newnewdata|>
#    mutate(age=as.factor(age))|>
    filter(age=="16–69")

graph4 <- ggplot(firstage4, aes(x=days, y=numtotal_atleast1dose))+
    geom_point(alpha=0.6)+
geom_line()+
    xlab("Date") +
    ylab("Number of New Vaccinations") +
    theme(text = element_text(size = 12))        

graph4

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 37)

#selected.1<- selected|>
# filter(age %in% c("0-4", "05-11", "12-17", "18-29", '30-39', '40-49', '50.59','60-69','70-79','80+'))

#selected.1<- selected|>
# filter(age %in% c("0-15", '0-17', '16-69', '18-69','70-74','75-79','80+'))

#selected.1

graph1.1 <- ggplot(newnewdata, aes(x=days, y=numtotal_atleast1dose, color=age))+
    geom_point(alpha=0.6)+
    geom_line()+
    facet_grid(rows = vars(age))+
    xlab("Date") +
    ylab("Number of New Vaccinations") +
    labs(color="Age Cathegory")+
    theme(text = element_text(size = 12))        

graph1.1